# Grecian Calendar Solver

## 0. Import Dependencies

In [34]:
import numpy as np

## 1. Create Layer Structure

In [306]:
x = np.nan
def make_layer(*rows, nrows = 4):
    if len(rows) > nrows:
        raise Exception('Number of rows in layer cannot exceed \'nrows\'')
        
    row_length = len(rows[0])
    arr = np.array(rows)
    
    if len(rows) < nrows:
        arr = np.concatenate((np.array([[np.nan]*row_length]*(nrows - len(rows))),
                              arr
                             ))
    return arr

In [294]:
def stack_layers(*layers):
    top = layers[0]
    for layer in layers[1:]:
        top = np.where(np.isnan(top), layer, top)
    return top

## 2. Create Solution Process

In [441]:
def solve(*layers, target = 42):
    n_layers = len(layers)
    layer_width = layers[0].shape[1]
    
    rotations = zip(*[x.flatten() for x in np.meshgrid(*[range(layer_width)]*(n_layers-1))])
    
    output = np.zeros((12**4, 4))
    i = 0
    for r in rotations:
        top_layer = stack_layers(*([np.roll(layers[i], r[i], 1) for i in range(n_layers-1)]))
        top_layer = stack_layers(top_layer, layers[-1])
        
        if all(np.sum(top_layer, 0) == target):
            print(top_layer, r, target)
            
        output[i] = [i, bimode_freq(np.sum(top_layer, 0)), mode_freq(np.sum(top_layer, 0))]
        i = i + 1
            
    return output

In [437]:
def mode_freq(arr):
    return max(np.unique(arr, return_counts = True)[1])

def bimode_freq(arr):
    freqs = np.unique(arr, return_counts = True)[1]
    if len(freqs) == 1:
        return freqs[0]
    return sum(np.partition(freqs, -2)[-2:])

## 3. Solve

In [381]:
layer1 = make_layer([3, x, 6, x, 10, x, 7, x, 15, x, 8, x])

layer2 = make_layer([4, x, 7, 15, x, x, 14, x, 9, x, 12, x],
                    [7, 3, x, 6, x, 11, 11, 6, 11, x, 6, 17])

layer3 = make_layer([5, x, 10, x, 8, x, 22, x, 16, x, 9, x],
                    [21, 6, 15, 4, 9, 18, 11, 26, 14, 1, 12, x],
                    [9, 13, 9, 7, 13, 21, 17, 4, 5, x, 7, 8],)

layer4 = make_layer([1, x, 9, x, 12, x, 6, x, 10, x, 10, x],
                    [3, 26, 6, x, 2, 13, 9, x, 17, 19, 3, 12],
                    [9, 20, 12, 3, 6, x, 14, 12, 3, 8, 9, x],
                    [7, x, 9, x, 7, 14, 11, x, 8, x, 16, 2])

layer5 = make_layer([2, 5, 10, 7, 16, 8, 7, 8, 8, 3, 4, 12],
                    [3, 3, 14, 14, 21, 21, 9, 9, 4, 4, 6, 6],
                    [8, 9, 10, 11, 12, 13, 14, 15, 4, 5, 6, 7],
                    [14, 11, 14, 14, 11, 14, 11, 14, 11, 11, 14, 11])
layer2

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [ 4., nan,  7., 15., nan, nan, 14., nan,  9., nan, 12., nan],
       [ 7.,  3., nan,  6., nan, 11., 11.,  6., 11., nan,  6., 17.]])

In [442]:
data = solve(layer1, layer2, layer3, layer4, layer5, target = 42)

[[ 1.  5.  9.  7. 12.  8.  6.  8. 10.  3. 10. 12.]
 [22. 26. 16. 14.  9. 13.  5.  9. 10. 19.  8. 12.]
 [11.  4. 14.  7. 15. 13. 21. 14. 15.  9.  9. 12.]
 [ 8.  7.  3. 14.  6.  8. 10. 11.  7. 11. 15.  6.]] (2, 1, 6, 0) 42


## 4. Explore Other Potential Solutions

In [444]:
data[np.argpartition(data[:,1], -20)[-20:]].astype(int)

array([[19066,     7,     6],
       [ 2952,     7,     6],
       [14655,     7,     5],
       [20016,     7,     4],
       [18909,     8,     6],
       [ 7127,     7,     6],
       [ 1818,     7,     4],
       [ 3240,     8,     6],
       [  215,     7,     6],
       [ 2390,     7,     5],
       [  373,     7,     5],
       [ 2304,     8,     6],
       [ 2664,     8,     6],
       [ 1776,     8,     6],
       [ 2088,    12,    12],
       [15139,     8,     6],
       [11513,     8,     6],
       [ 3973,     8,     7],
       [ 5858,     8,     6],
       [12770,     8,     7]])

In [371]:
small_inds = sds[np.argpartition(sds[:,1], 10)[:10]][:,0]
small_inds.astype(int)

array([ 7127, 12770,  2088, 17901,  9628, 15591,  3973, 18909,  7743,
        2664])

small sds at indices 7127, 3254, 16462, 14159.

In [445]:
r = list(zip(*[x.flatten() for x in np.meshgrid(*[range(12)]*(5-1))]))[11513]
r

(7, 6, 11, 5)

In [446]:
top_layer = stack_layers(*([np.roll(layers[i], r[i], 1) for i in range(5-1)]))
top_layer = stack_layers(top_layer, layers[-1])
top_layer

array([[ 2., 10., 10., 10., 16.,  1.,  7.,  9.,  8., 12.,  4.,  6.],
       [ 3., 10., 19.,  8., 12., 22., 26., 16.,  4.,  9., 13.,  5.],
       [14., 15.,  9.,  9., 12., 11.,  4., 14.,  7., 15.,  6., 21.],
       [11.,  7., 11., 15.,  6.,  8.,  7.,  3., 11.,  6.,  8., 10.]])

In [447]:
test = np.sum(top_layer,0)
test

array([30., 42., 49., 42., 46., 42., 44., 42., 30., 42., 31., 42.])

In [422]:
test = np.array([5, 10, 3, 20, 10, 3, 3])
np.partition(np.unique(test, return_counts = True)[1], 2)[-2:]

array([2, 3], dtype=int64)